https://huggingface.co/docs/transformers/en/training

https://huggingface.co/datasets/Yelp/yelp_review_full

https://huggingface.co/google-bert/bert-base-cased

In [1]:
import os

## dataset

In [2]:
from datasets import load_dataset

In [3]:
dataset = load_dataset("yelp_review_full")

In [4]:
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [5]:
from transformers import AutoTokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased", clean_up_tokenization_spaces=True)

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [8]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [9]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))

In [10]:
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

## train

In [11]:
from transformers import AutoModelForSequenceClassification

In [12]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5, device_map="cuda")

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [13]:
def evaluate(s: str):
    inputs = tokenizer(s, return_tensors="pt")
    inputs = inputs.to("cuda")
    model_outputs = model(**inputs)
    return model_outputs[0]

In [14]:
evaluate("that fish is super tasty, I liked it so much. Highly recommend")

tensor([[0.6354, 0.1063, 0.0342, 0.0335, 1.0170]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [15]:
evaluate("the worst fish I've ever eaten, can't imaging how they cooked")

tensor([[ 0.5430,  0.0321, -0.0736,  0.0064,  1.0093]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [16]:
evaluate("not bad, I eated it but it could be better")

tensor([[0.6386, 0.1061, 0.0027, 0.0264, 0.9431]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

## train

In [17]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [18]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [19]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer", 
    eval_strategy="epoch",
    report_to="none",
)

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.097396,0.516000
2,No log,1.037490,0.559000
3,No log,1.021638,0.581000


TrainOutput(global_step=375, training_loss=1.0150188802083333, metrics={'train_runtime': 253.5026, 'train_samples_per_second': 11.834, 'train_steps_per_second': 1.479, 'total_flos': 789354427392000.0, 'train_loss': 1.0150188802083333, 'epoch': 3.0})

## check

In [22]:
def evaluate2(s: str):
    inputs = tokenizer(s, return_tensors="pt")
    inputs = inputs.to("cuda")
    model_outputs = model(**inputs)
    return model_outputs[0]

5 logits are 1-start up to 5-star

In [23]:
evaluate2("that fish is super tasty, I liked it so much. Highly recommend")

tensor([[-1.7722, -1.8112, -0.7527,  2.1802,  2.7354]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [24]:
evaluate2("the worst fish I've ever eaten, can't imaging how they cooked")

tensor([[ 3.2862,  0.8892, -0.7286, -1.5774, -1.1951]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [25]:
evaluate2("not bad, I eated it but it could be better")

tensor([[-0.8633,  1.6398,  2.1891, -0.3914, -1.9721]], device='cuda:0',
       grad_fn=<AddmmBackward0>)